In [ ]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 49.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.6.9 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.2.1 requires jax>=0.4.8, but you have jax 0.3.25 which is incompatible.


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14693528306915591757
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14328594432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4142470869565993200
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
#imported libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np
from sklearn.model_selection import train_test_split
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from keras.utils import to_categorical
import matplotlib.pyplot as plt 
import keras
from time import time
from keras.layers.core import Activation, Dropout, Dense, Masking
#padding and tokenization
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras.optimizers
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#load the dataset
dataset1_path = '/content/drive/MyDrive/Colab Notebooks/players_preprocessed_new_data.csv'
dataset2_path = '/content/drive/MyDrive/Colab Notebooks/preprocessed_airlines_data (1).csv'

tweet_airline_sentiment = pd.read_csv(dataset2_path)
tweet_player_sentiment = pd.read_csv(dataset1_path)


In [9]:
# Change the column names
new_column_names = ['Unnamed: 0','text', 'airline_sentiment',]
tweet_player_sentiment.columns = new_column_names

# Append dataset 10-14 to dataset 1-10
sentiment_data = tweet_airline_sentiment.append(tweet_player_sentiment)

# Optional: Reset the index of the merged dataset
sentiment_data = sentiment_data.reset_index(drop=True)

# Rename a single column
sentiment_data.rename(columns={'airline_sentiment': 'sentiment'}, inplace=True)

sentiment_data = sentiment_data.drop('Unnamed: 0', axis=1)

# Save the merged dataset to a new CSV file
sentiment_data.to_csv('Sentiment_dataset.csv', index=False)



<ipython-input-9-b80c3362e6d9>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentiment_data = tweet_airline_sentiment.append(tweet_player_sentiment)


In [10]:
sentiment_data

,text,sentiment
0,said,neutral
1,plus youve added commercial experience tacky,positive
2,didnt today must mean need take another trip,neutral
3,really aggressive blast obnoxious entertainmen...,negative
4,really big bad thing,negative
...,...,...
333925,trophy best playstyle serie problem,positive
333926,excited,positive
333927,sleep well dont worry cuz ur team fucking shit,negative
333928,google daniel twitch franco jayz,neutral


In [11]:
# Encode the sentiment labels
label_encoder = LabelEncoder()
sentiment_data['sentiment'] = label_encoder.fit_transform(sentiment_data['sentiment'])

In [13]:
# train, val, test split

x_train, xtest, y_train, ytest = train_test_split(sentiment_data.text.values,sentiment_data.sentiment,stratify=sentiment_data.sentiment.values, test_size=0.3,random_state=15)
y_train = to_categorical(y_train)
x_val = xtest[0:25000]
y_val = to_categorical(ytest[0:25000])
x_test = xtest[25000:]
y_test = ytest[25000:]

# Convert non-string values to strings
sentiment_data.text = sentiment_data.text.astype(str)
x_train = x_train.astype(str)
x_val = x_val.astype(str)
x_test = x_test.astype(str)

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentiment_data.text.values)


X_train = tokenizer.texts_to_sequences(x_train)
X_val = tokenizer.texts_to_sequences(x_val)
X_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 100
#padding
X_train = pad_sequences(X_train, padding='pre', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='pre', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='pre', maxlen=maxlen)

word_index = tokenizer.word_index

In [14]:
#import glove embeddings

embeddings_index = {}
f = open(os.path.join( '/content/drive/MyDrive/Colab Notebooks/word_embeddings.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [1]:
import tensorflow as tf

# Check if GPU is available
if tf.test.gpu_device_name():
    print('GPU available')
else:
    print('GPU not available')

# Set TensorFlow to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


GPU available


In [35]:
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from time import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

embedding_dim =100
num_classes = 3
# Define the function to create the model
def create_model(filters=516, kernel_size=3, pool_size=3, dense_units=64, dropout_rate=0.4, learning_rate=0.01):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(units=dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [16]:
# Define the hyperparameters to search
param_dist = {
    'filters': [256, 512],
    'kernel_size': [3, 5],
    'dense_units': [32, 64]
}

# Create the KerasClassifier model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=128, verbose=0)

# Perform randomized search
n_iter_search = 2  # Set a lower number of iterations
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=n_iter_search)

# Fit the randomized search
start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates" % ((time() - start), n_iter_search))

# Get the best parameters and model
best_params = random_search.best_params_
best_model = random_search.best_estimator_

# Train the best model
best_model.fit(X_train, y_train, validation_data=(X_val, y_val))

<ipython-input-16-5afcab744667>:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=128, verbose=0)


RandomizedSearchCV took 2240.35 seconds for 2 candidates


In [18]:
print(random_search.best_params_)

{'kernel_size': 3, 'filters': 256, 'dense_units': 64}


In [36]:
model.fit(X_train,y_train,epochs=10,batch_size=800,validation_data=(X_val,y_val))

In [50]:

from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

# Convert target labels to one-hot encoding
y_test_encoded = to_categorical(y_test, num_classes=3)
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.7619414925575256


In [52]:
# Assuming you have a trained KerasClassifier model assigned to the variable `model`

keras_model = model.model
keras_model.save("cnn-model.h5")
